<h3>Import Packages<h3>

In [22]:
import pandas as pd
from sqlalchemy.exc import IntegrityError
from sqlalchemy import create_engine
import numpy as np
import tejapi

<h3>Import Stock id<h3>

In [23]:
df= pd.read_csv('stockid.csv')

sotck_id = np.array(df)
stockid_list=sotck_id.tolist()
print(sotck_id)

[[1101]
 [1102]
 [1103]
 ...
 [9958]
 [9960]
 [9962]]


<h3>Connect MySQL<h3>

In [24]:
def connect_sql(database,echo):
    engine = create_engine("mysql+pymysql://root:")
    return engine
engine = connect_sql('twse',False)

<h3>Import Stock Data From TEJ_api<h3>

In [25]:
tejapi.ApiConfig.api_key = "api_key"

<h3>Define Funition That Inserts Data to MySQL<h3>

In [26]:

def insertsql(stockid):
    coid=stockid
    data = tejapi.get('TWN/APRCD1',coid=coid,mdate={'gt':'2006-12-31'},opts={'columns':['mdate','open_adj','close_adj','volume','high_adj','low_adj']})
    create = '''create table   '''
    stockid = 'id' + coid
    text = '''  ( mdate                datetime
    ,open_adj             decimal(8,2)
    ,close_adj            decimal(8,2)
    ,high_adj	          decimal(8,2)
    ,low_adj	          decimal(8,2)
    ,volume	              decimal(9,0)
);'''
    sqltext = create + stockid + text
    drop = 'drop table IF EXISTS' +' '+ stockid +';'
    try:
        engine.execute(drop)
        engine.execute(sqltext)
        
    except IntegrityError:
        print('table already exists')
        
    try:
        data.to_sql(name=stockid, con=engine, if_exists='append', index=False)
    
    except IntegrityError:
        print('catch duplicate data')

<h3>Execute the Loop to Insert Data in MySQL for Each Stock id<h3>

In [ ]:
for i in range(0,1877,1):
    id=stockid_list[i]
    id=id[0]
    id=str(id)
    insertsql(id)
    print(id)